In [ ]:
!pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.7/383.7 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/325.2 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 1.5 MB/s eta 0:00:00


In [ ]:
import openai
import json
import time
from IPython.display import display, Markdown

from google.colab import userdata
# Initialize the OpenAI client
client = openai.OpenAI(api_key =userdata.get('OPEN_AI_KEY'))

In [ ]:
def make_api_call(messages, max_tokens, is_final_answer=False):
    for attempt in range(3):
        try:
            response = client.chat.completions.create(
                model="gpt-4o",  # Using GPT-3.5 Turbo, adjust as needed
                messages=messages,
                max_tokens=3000,
                temperature=0.2,
                response_format={"type": "json_object"}
            )
            return json.loads(response.choices[0].message.content)
        except Exception as e:
            if attempt == 2:
                if is_final_answer:
                    return {"title": "Error",
                            "content": f"Failed to generate final answer after 3 attempts. Error: {str(e)}"}
                else:
                    return {"title": "Error", "content": f"Failed to generate step after 3 attempts. Error: {str(e)}",
                            "next_action": "final_answer"}
            time.sleep(1)  # Wait for 1 second before retrying

In [ ]:
def generate_response_steps(prompt):
    messages = [
        {"role": "system", "content": """You are an expert AI assistant with advanced reasoning capabilities. Your task is to provide detailed, step-by-step explanations of your thought process. For each step:

1. Provide a clear, concise title describing the current reasoning phase.
2. Elaborate on your thought process in the content section.
3. Decide whether to continue reasoning or provide a final answer.

Response Format:
Use JSON with keys: 'title', 'content', 'next_action' (values: 'continue' or 'final_answer')

Key Instructions:
- Employ at least 5 distinct reasoning steps.
- Acknowledge your limitations as an AI and explicitly state what you can and cannot do.
- Actively explore and evaluate alternative answers or approaches.
- Critically assess your own reasoning; identify potential flaws or biases.
- When re-examining, employ a fundamentally different approach or perspective.
- Utilize at least 3 diverse methods to derive or verify your answer.
- Incorporate relevant domain knowledge and best practices in your reasoning.
- Quantify certainty levels for each step and the final conclusion when applicable.
- Consider potential edge cases or exceptions to your reasoning.
- Provide clear justifications for eliminating alternative hypotheses.


Example of a valid JSON response:
```json
{
    "title": "Initial Problem Analysis",
    "content": "To approach this problem effectively, I'll first break down the given information into key components. This involves identifying...[detailed explanation]... By structuring the problem this way, we can systematically address each aspect.",
    "next_action": "continue"
}```
"""},
        {"role": "user", "content": prompt},
        {"role": "assistant",
         "content": "Thank you! I will now think step by step following my instructions, starting at the beginning after decomposing the problem."}
    ]

    steps = []
    step_count = 1
    total_thinking_time = 0

    while True:
        start_time = time.time()
        step_data = make_api_call(messages, 300)
        end_time = time.time()
        thinking_time = end_time - start_time
        total_thinking_time += thinking_time

        steps.append((f"Step {step_count}: {step_data['title']}", step_data['content'], thinking_time))

        messages.append({"role": "assistant", "content": json.dumps(step_data)})

        if step_data['next_action'] == 'final_answer':
            break

        step_count += 1

        # Yield after each step for Streamlit to update
        yield steps, None  # We're not yielding the total time until the end

    # Generate final answer
    messages.append({"role": "user", "content": "Please provide the final answer based on your reasoning above."})

    start_time = time.time()
    final_data = make_api_call(messages, 200, is_final_answer=True)
    end_time = time.time()
    thinking_time = end_time - start_time
    total_thinking_time += thinking_time

    steps.append(("Final Answer", final_data['content'], thinking_time))

    yield steps, total_thinking_time

In [ ]:
import html
from IPython.display import display, Markdown, HTML
def generate_response(prompt):
    # ... [previous code remains unchanged] ...
    content = None
    for steps, total_thinking_time in generate_response_steps(prompt):
        for i, (title, content, thinking_time) in enumerate(steps):
            if title.startswith("Final Answer"):
                display(Markdown(f"### {title}"))
                display(HTML(content.replace('\n', '<br>')))
            else:
                display(Markdown(f"### {title}"))
                display(HTML(content.replace('\n', '<br>')))
            display(Markdown(f"*Thinking time: {thinking_time:.2f} seconds*"))

        # Only show total time when it's available at the end
        if total_thinking_time is not None:
            display(Markdown(f"**Total thinking time: {total_thinking_time:.2f} seconds**"))
    return content

In [ ]:
user_query = "generate 10 questions in calculus for high school students"
generate_response(user_query)

### Step 1: Understanding the Requirements

*Thinking time: 1.70 seconds*

### Step 1: Understanding the Requirements

*Thinking time: 1.70 seconds*

### Step 2: Identifying Key Topics in High School Calculus

...

### Step 6: Finalizing the Set of Questions

*Thinking time: 1.66 seconds*

### Final Answer

*Thinking time: 8.44 seconds*

**Total thinking time: 33.48 seconds**

In [ ]:
!pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.8 MB/s eta 0:00:00


In [ ]:
import PyPDF2
def extract_text_from_pdf(pdf_file):
    """Extracts and returns the full text from a PDF file."""
    try:
        reader = PyPDF2.PdfReader(pdf_file)
        full_text = ""
        for page in reader.pages:
            full_text += page.extract_text() + "\n"
        return full_text
    except Exception as e:
        return ""

In [ ]:
!pip install langchain langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 38.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 407.2/407.2 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 3.1 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0


In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage
import re
def process_exam_with_openai(full_text, chat_model):
    """Processes the extracted text to identify and format MCQs using OpenAI."""
    # Define the system prompt
    system_prompt = """
    You are an advanced AI assistant that extracts and processes multiple-choice questions (MCQs) from text.
    Please extract the following for each identified question:

    1. The question stem.
    2. Four answer options (A, B, C, D).
    3. Metadata including:
       - Topic
       - Subtopic
       - Cognitive Level
       - Expected Solution Time

    Use the following JSON format for each question:
    {
        "question": "",
        "options": ["", "", "", ""],
        "metadata": {
            "topic": "",
            "subtopic": "",
            "cognitive_level": "",
            "expected_solution_time": ""
        }
    }
    return only the JSON output with nothing else and ensure that the whole response is a proper json file that can be interpreted.
    """

    # Prepare the message with the extracted text
    messages = [
        SystemMessage(content=system_prompt),
        HumanMessage(content=f"Extract and format the MCQs from this text:\n{full_text}")
    ]

    # Get response from OpenAI
    response = chat_model(messages)
    response_content = response.content.strip()
    response_content = re.sub(r'```json\n', '', response_content)
    response_content = re.sub(r'\n```', '', response_content)
    #response_content = ['[',response_content,']']

    # Parse and display the formatted questions
    try:
        formatted_questions = json.loads(response_content)
        return formatted_questions
    except json.JSONDecodeError:
        print("Error: Unable to parse the JSON output from OpenAI.")
        print(f"Response Content: {response_content}")
        return []


def process_outline_with_openai(full_text, chat_model):
    """Processes the extracted text to identify and format MCQs using OpenAI."""
    # Define the system prompt
    system_prompt = """
    You are an advanced AI assistant that extracts and processes Course Outline from text.
    Please extract the following for each identified question:

    1. Main topics
    2. suptopics of the main topics
    3. Learning outcomes and brief content

    **output format**:
     topic:
        -subtopic:
            -learning outcomes:
    """

    # Prepare the message with the extracted text
    messages = [
        SystemMessage(content=system_prompt),
        HumanMessage(content=f"Extract and format the MCQs from this text:\n{full_text}")
    ]

    # Get response from OpenAI
    response = chat_model(messages)
    response_content = response.content.strip()
    response_content = re.sub(r'```json\n', '', response_content)
    response_content = re.sub(r'\n```', '', response_content)
    #response_content = ['[',response_content,']']

    return response_content

In [ ]:
chat_model = ChatOpenAI(model="gpt-4o", temperature=0.2,openai_api_key = userdata.get('OPEN_AI_KEY'))

In [ ]:
example_exam_file = 'example_exam.pdf'
example_exam = extract_text_from_pdf(example_exam_file)
example_exam = process_exam_with_openai(example_exam, chat_model)

In [ ]:
user_query = f"generate 10 questions like the questions in that exam \n {example_exam}.\n don't use the same questions but create with the same structure, variance and topics"

In [ ]:
generate_response(user_query)

### Step 1: Understanding the Task Requirements

*Thinking time: 1.92 seconds*

### Step 1: Understanding the Task Requirements

*Thinking time: 1.92 seconds*

### Step 2: Identifying Key Components of Example Questions

...

### Step 6: Finalizing and Reviewing Generated Questions

*Thinking time: 1.04 seconds*

### Final Answer

*Thinking time: 17.51 seconds*

**Total thinking time: 26.15 seconds**

In [ ]:
def process_outline_with_openai(full_text, chat_model):
    """Processes the extracted text to identify and format MCQs using OpenAI."""
    # Define the system prompt
    system_prompt = """
    You are an advanced AI assistant that extracts and processes Course Outline from text.
    Please extract the following for each identified question:

    1. Main topics
    2. suptopics of the main topics
    3. Learning outcomes and brief content

    **output format**:
     topic:
        -subtopic:
            -learning outcomes:
    """

    # Prepare the message with the extracted text
    messages = [
        SystemMessage(content=system_prompt),
        HumanMessage(content=f"Extract and format the MCQs from this text:\n{full_text}")
    ]

    # Get response from OpenAI
    response = chat_model(messages)
    response_content = response.content.strip()
    response_content = re.sub(r'```json\n', '', response_content)
    response_content = re.sub(r'\n```', '', response_content)
    #response_content = ['[',response_content,']']

    return response_content

In [ ]:
outline_file = 'outline.pdf'
outline = extract_text_from_pdf(outline_file)
outline = process_outline_with_openai(outline, chat_model)

In [ ]:
question_prompt = f"""
Using the provided sample exam as the sole reference, generate 25 challenging and high-quality question stems for a multiple-choice exam.

### Sample Exam:
{example_exam}

- Ensure that the generated questions maintain the same level of difficulty, complexity, and quality as the sample exam.
- Use the sample exam as a style reference for question format, structure, and type (e.g., definitions, calculations, applications, reasoning).
- Include a variety of question types to assess different cognitive levels (e.g., understanding, application, analysis).
- Incorporate a mix of very hard and creative questions, consistent with the sample exam's difficulty.
- Focus only on generating the question stems, without options or answers.
"""


In [ ]:
print(example_exam)

[{'question': 'If x = sec y, where y ∈]π/2, π[, then dx/dy = ...', 'options': ['-x sqrt(x^2 - 1)', 'x sqrt(x^2 - 1)', '-x sqrt(x^2 + 1)', 'x sqrt(x^2 + 1)'], 'metadata': {'topic': 'Calculus', 'subtopic': 'Differentiation', 'cognitive_level': 'Application', 'expected_solution_time': '2 minutes'}}, {'question': 'lim x→1 (e^x - e)/(x - 1) = ...', 'options': ['e', '-e', '1', '-1'], 'metadata': {'topic': 'Calculus', 'subtopic': 'Limits', 'cognitive_level': 'Understanding', 'expected_solution_time': '1 minute'}}, {'question': 'If lim x→0 (ln(x + 1))^k/x = 4, then k = ...', 'options': ['16', '4', '8', '2'], 'metadata': {'topic': 'Calculus', 'subtopic': 'Limits', 'cognitive_level': 'Application', 'expected_solution_time': '2 minutes'}}, {'question': 'If f(x) = ∫(1 - (ln x)^2)/x dx, where f(1) = 0, then f(e) = ...', 'options': ['2/3', '-2/3', '1/3', '-1/3'], 'metadata': {'topic': 'Calculus', 'subtopic': 'Integration', 'cognitive_level': 'Analysis', 'expected_solution_time': '3 minutes'}}, {'que

In [ ]:
question_stems = generate_response(question_prompt)

### Step 1: Understanding the Task Requirements

*Thinking time: 1.44 seconds*

### Step 1: Understanding the Task Requirements

*Thinking time: 1.44 seconds*

### Step 2: Analyzing the Sample Exam

...

### Step 6: Question Stem Generation

*Thinking time: 8.01 seconds*

### Final Answer

*Thinking time: 11.19 seconds*

**Total thinking time: 27.54 seconds**

In [ ]:
question_stems

"Here are 25 challenging and high-quality question stems for a multiple-choice exam, inspired by the sample exam:\n\n1. If f(x) = ln(x^2 + 1), then the second derivative f''(x) is...\n2. Evaluate the limit: lim x→0 (sin(2x)/x) = ...\n3. If y = tan^(-1)(x^2), find dy/dx when x = 1.\n4. Determine the convergence of the series ∑ (n=1 to ∞) (1/n^2).\n5. If f(x) = e^(3x) and g(x) = ln(x), find (f ∘ g)'(x).\n6. Solve the differential equation: dy/dx = y^2 - 4.\n7. If the function f is continuous on [0, 2] and differentiable on (0, 2), and f(0) = 1, f(2) = 3, find f'(c) for some c in (0, 2).\n8. If y = x^x, find dy/dx.\n9. Evaluate the integral: ∫ (0 to π/2) sin^3(x) cos^2(x) dx.\n10. If the function g(x) = x^3 - 3x + 2 has a local maximum at x = a, find the value of a.\n11. Determine the radius of convergence for the power series ∑ (n=0 to ∞) (x^n/n!).\n12. If f(x) = x^2 + 2x + 1, find the inverse function f^(-1)(x).\n13. If y = cos(x) + sin(x), find the maximum value of y on the interval [0

In [ ]:
options_prompt = f"""
Create four high quality answer options (A, B, C, D) for all the following questions stems:
{question_stems}



- YOU MUST CREATE FOR ALL THE QUESTIONS
- You must output the questions with the options and the correct option
- One correct answer and three high-quality distractors.
- Distractors should be plausible and designed to reveal common misconceptions or errors.
- Vary the difficulty of the options: at least one should be very close to the correct answer, and others should include common mistakes.
- Use the example exam as a reference for the complexity and style of options:
###Sample Exam:
{example_exam}
"""

In [ ]:
questions_list = generate_response(options_prompt)

### Step 1: Understanding the Task Requirements

*Thinking time: 1.70 seconds*

### Step 1: Understanding the Task Requirements

*Thinking time: 1.70 seconds*

### Step 2: Analyzing the Sample Exam for Style and Complexity

...

### Step 28: Creating Options for Question 25

*Thinking time: 3.06 seconds*

### Final Answer

*Thinking time: 22.17 seconds*

**Total thinking time: 111.43 seconds**

In [ ]:
questions_list

"Here are the questions with their respective answer options, including the correct answer and plausible distractors:\n\n1. If f(x) = ln(x^2 + 1), then the second derivative f''(x) is...\n   - A) (2 - 2x^2)/(x^2 + 1)^2  [Correct]\n   - B) (2x^2 - 2)/(x^2 + 1)^2\n   - C) (2x)/(x^2 + 1)\n   - D) (2x^2 + 2)/(x^2 + 1)^2\n\n2. Evaluate the limit: lim x→0 (sin(2x)/x) = ...\n   - A) 2  [Correct]\n   - B) 1\n   - C) 0\n   - D) ∞\n\n3. If y = tan^(-1)(x^2), find dy/dx when x = 1.\n   - A) 1  [Correct]\n   - B) 2\n   - C) 1/2\n   - D) 0\n\n4. Determine the convergence of the series ∑ (n=1 to ∞) (1/n^2).\n   - A) Converges  [Correct]\n   - B) Diverges\n   - C) Converges conditionally\n   - D) Oscillates\n\n5. If f(x) = e^(3x) and g(x) = ln(x), find (f ∘ g)'(x).\n   - A) 3x  [Correct]\n   - B) 3/x\n   - C) 3x^2\n   - D) e^(3x)/x\n\n6. Solve the differential equation: dy/dx = y^2 - 4.\n   - A) y = 2 tanh(2x + C)  [Correct]\n   - B) y = 2 coth(2x + C)\n   - C) y = 2 sech(2x + C)\n   - D) y = 2 csch(

In [ ]:
analysis_prompt = f"""
Compare the following generated exam with the provided sample exam, and provide a detailed analysis of the following aspects:

### Generated Exam:
{questions_list}

### Sample Exam:
{example_exam}

1. **Difficulty Comparison**:
   - Analyze the difficulty of each question in the generated exam and compare it to the equivalent questions in the sample exam.
   - Classify each question as easy, medium, hard, or very hard, and explain any differences in difficulty level between the generated and sample exams.

2. **Topic Comparison**:
   - Identify the main topic or concept that each question in the generated exam is testing.
   - Compare the topic coverage of the generated exam to the sample exam to see if the same topics and concepts are being tested.
   - Highlight any topics that are missing or overrepresented in the generated exam relative to the sample exam.

3. **Question Variety and Style**:
   - Assess the variety of question types (definition, calculation, application, reasoning) in both exams.
   - Determine if the generated exam matches the style and type of questions (in terms of wording, structure, and cognitive level) found in the sample exam.

4. **Suggestions for Improvement**:
   - Provide recommendations for improving the generated exam to match the quality, difficulty, and topic coverage of the sample exam more closely.
   - Suggest any necessary adjustments in question phrasing, difficulty balance, or topic distribution to ensure the generated exam meets the same standards as the sample exam.

Return a comprehensive analysis along with actionable insights for refining the generated exam.
"""


In [ ]:
analysis = generate_response(analysis_prompt)

### Step 1: Initial Problem Analysis

*Thinking time: 1.50 seconds*

### Step 1: Initial Problem Analysis

*Thinking time: 1.50 seconds*

### Step 2: Difficulty Comparison

...

### Step 5: Suggestions for Improvement

*Thinking time: 3.49 seconds*

### Final Answer

*Thinking time: 3.15 seconds*

**Total thinking time: 19.32 seconds**

In [ ]:
analysis_prompt = f"""
Compare the following generated exam with the provided sample exam, and provide a detailed analysis of the questions based on their difficulty and topic for both exams.

### Generated Exam:
{questions_list}

### Sample Exam:
{example_exam}

1. **Difficulty Analysis**:
   - Classify the difficulty of each question in both the generated exam and the sample exam as **easy**, **medium**, or **hard**.
   - Compare the difficulty of the questions in both exams and return the question numbers in the following format:

   **Generated Exam:**
   - **Easy questions**: [list question numbers]
   - **Medium questions**: [list question numbers]
   - **Hard questions**: [list question numbers]

   **Sample Exam:**
   - **Easy questions**: [list question numbers]
   - **Medium questions**: [list question numbers]
   - **Hard questions**: [list question numbers]

2. **Topic Analysis**:
   - Identify the main topic of each question in both the generated exam and the sample exam and categorize them based on the course topics.
   - Return the question numbers corresponding to each topic for both exams in the following format:

   **Generated Exam:**
   - **<Topic 1>**: [list question numbers]
   - **<Topic 2>**: [list question numbers]

   **Sample Exam:**
   - **<Topic 1>**: [list question numbers]
   - **<Topic 2>**: [list question numbers]

Ensure the analysis is structured and concise, clearly indicating the difficulty level and topic alignment for both the generated and sample exams.
"""


In [ ]:
analysis = generate_response(analysis_prompt)

### Step 1: Understanding the Task Requirements

*Thinking time: 1.57 seconds*

### Step 1: Understanding the Task Requirements

*Thinking time: 1.57 seconds*

### Step 2: Initial Difficulty Assessment for Generated Exam

...

### Step 10: Final Analysis and Conclusion

*Thinking time: 7.61 seconds*

### Final Answer

*Thinking time: 9.30 seconds*

**Total thinking time: 37.93 seconds**

In [ ]:
modification_prompt = f"""
Based on the following analysis of the generated exam and sample exam, modify the generated exam to match the same distribution of difficulty (easy, medium, hard) and topic coverage as the sample exam.

### Generated Exam:
{questions_list}

### Analysis:
{analysis}

### Instructions:
1. **Match Difficulty Distribution**:
   - Adjust the generated exam to match the number of easy, medium, and hard questions as seen in the sample exam.
   - Modify the difficulty level of questions by rephrasing or changing the complexity, while preserving the core concept being tested.

2. **Match Topic Distribution**:
   - Ensure that the same topics are covered in the generated exam as in the sample exam.
   - Add or modify questions in the generated exam to match the topic distribution of the sample exam, ensuring a similar number of questions for each topic.

3. **Return the Modified Exam**:
   - Once the difficulty and topic distributions match the sample exam, return the fully modified generated exam.

Ensure that only difficulty level and topic coverage are adjusted, with no other changes to question types or structure.
"""


In [ ]:
improved_questions = generate_response(modification_prompt)

### Step 1: Understanding the Problem Requirements

*Thinking time: 2.12 seconds*

### Step 1: Understanding the Problem Requirements

*Thinking time: 2.12 seconds*

### Step 2: Analyzing Difficulty Distribution

...

### Step 6: Implementing Changes to the Generated Exam

*Thinking time: 2.84 seconds*

### Final Answer

*Thinking time: 26.42 seconds*

**Total thinking time: 42.25 seconds**

In [ ]:
example_exam

[{'question': 'If x = sec y, where y ∈]π/2, π[, then dx/dy = ...',
  'options': ['-x sqrt(x^2 - 1)',
   'x sqrt(x^2 - 1)',
   '-x sqrt(x^2 + 1)',
   'x sqrt(x^2 + 1)'],
  'metadata': {'topic': 'Calculus',
   'subtopic': 'Differentiation',
   'cognitive_level': 'Application',
   'expected_solution_time': '2 minutes'}},
 {'question': 'lim x→1 (e^x - e)/(x - 1) = ...',
  'options': ['e', '-e', '1', '-1'],
  'metadata': {'topic': 'Calculus',
   'subtopic': 'Limits',
   'cognitive_level': 'Understanding',
   'expected_solution_time': '1 minute'}},
 {'question': 'If lim x→0 (ln(x + 1))^k/x = 4, then k = ...',
  'options': ['16', '4', '8', '2'],
  'metadata': {'topic': 'Calculus',
   'subtopic': 'Limits',
   'cognitive_level': 'Application',
   'expected_solution_time': '2 minutes'}},
 {'question': 'If f(x) = ∫(1 - (ln x)^2)/x dx, where f(1) = 0, then f(e) = ...',
  'options': ['2/3', '-2/3', '1/3', '-1/3'],
  'metadata': {'topic': 'Calculus',
   'subtopic': 'Integration',
   'cognitive_level

In [ ]:
prompt = f"take this json exam and return a regular formatted exam {example_exam}"
make_api_call(prompt,500)

{'title': 'Error',
 'content': 'Failed to generate step after 3 attempts. Error: Error code: 400 - {\'error\': {\'message\': "Invalid type for \'messages\': expected an array of objects, but got a string instead.", \'type\': \'invalid_request_error\', \'param\': \'messages\', \'code\': \'invalid_type\'}}',
 'next_action': 'final_answer'}